In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Device Characteristics

In [ ]:
device_char = pd.read_csv('./device_char.csv')
display(device_char)

# 0. Accuracy

In [ ]:
df = pd.read_csv("./metrics_stat_testbed.csv")
accuracies = []
rounds = []
for i in range(60):
    rows = df[i*4:i*4+4]
    acc = np.mean(rows['accuracy'])
    accuracies.append(acc)
    rounds.append(i)
    
ax = plt.figure(figsize=(8, 6), dpi=80)
ax = plt.plot(rounds, accuracies)
ax = plt.xlabel("round")
ax = plt.ylabel("accuracy")

# 1. Training time

### 1.1 Average Training time

In [ ]:
df = pd.read_csv("./metrics_stat_testbed.csv")
device_ids = df["client_id"].unique()

In [ ]:
device_ids

In [ ]:
devices = ['Nexus 5x', 'Nexus 6P', "Nokia 1", "Nexus 5"]
devices

In [ ]:
avg_train_time = {}
device_data = {}
for device in device_ids:
    data = df[df["client_id"] == device]
    device_id = data.iloc[0]["client_id"]
    if device_id == "29810585-de37-4bee-888e-6d7f453c04a6_1":
        device_data["Nexus 5x"] = data["total_train_time"]/60000
        
    elif device_id == "c2515b04-a8a3-413c-b973-3685cab84c06_1":
        device_data["Nexus 6P"] = data["total_train_time"]/60000
        
    elif device_id == "e127265f-912b-4b35-bcee-d89bdba012b3_1":
        device_data["Nokia 1"] = data["total_train_time"]/60000
        
    elif device_id == "a6320ea7-31ca-4ed8-9c54-cf4632a9a074_1":
        device_data["Nexus 5"] = data["total_train_time"]/60000

for device in devices:
    avg_train_time[device] = device_data[device].mean()
    
ax = sns.barplot(x=list(avg_train_time.keys()), y=list(avg_train_time.values()))
ax.set_ylabel("Training time per round (minutes)")
ax.set_xlabel("Device")

### 1.2 Confidence Intervals

In [ ]:
# z=1.96 for 95% confidence intervals
def plot_confidence_interval(x, values, z=1.96, color='#2187bb', horizontal_line_width=0.2):
    mean = statistics.mean(values)
    stdev = statistics.stdev(values)
    confidence_interval = z * stdev / sqrt(len(values))

    left = x - horizontal_line_width / 2
    top = mean - confidence_interval
    right = x + horizontal_line_width / 2
    bottom = mean + confidence_interval
    plt.plot([x, x], [top, bottom], color=color)
    plt.plot([left, right], [top, top], color=color)
    plt.plot([left, right], [bottom, bottom], color=color)
    plt.plot(x, mean, 'o', color='#f44336')

    return mean, confidence_interval

In [ ]:
import matplotlib
import statistics
from math import sqrt

matplotlib.rcParams.update({'font.size': 14})
plt.xticks([1, 2, 3, 4], devices)
for i, device in enumerate(devices):
    plot_confidence_interval(i+1, device_data[device])

plt.title('95% Confidence Interval of average per round training time')
plt.xlabel('Device')
plt.ylabel('time')
plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 14})
plt.xticks([1, 2], devices[:2])
for i, device in enumerate(devices[:2]):
    plot_confidence_interval(i+1, device_data[device])

In [ ]:
for device in devices:
    std = np.std(device_data[device])
    print(f'std for {device} = {std}')

# 2. Memory Profile

### 2.1 Time series

In [ ]:
memory_info = {}
for device in ["Nexus 5x", "Nexus 6P", "Nexus 5"]: # ADD "Nokia 1"
    file = f'./Expt/{device}_memLog.csv'
    memory_info[device] = pd.read_csv(file)

def plot(to_plot):
    for device in list(memory_info.keys()):
        data = memory_info[device][to_plot].to_numpy()
        sns.lineplot(data=data, label=device)
        plt.ylabel(to_plot)
    plt.show()

plot("Active_Memory(MB)")
plot("org.openmined.syft.demo_pss(MB)")
plot("Cached_Memory(MB)")
plot("Free_Memory(MB)")
plot("pressure_pss(MB)")
plot("SwapCached_Memory(MB)")

### 2.2 Average

In [ ]:
# Average active memory for all devices
# Average cached memory for all devices
# Average free memory for all devices

# 3. CPU Profile

In [ ]:
def calculate_cpu_percentage(prev_total, prev_idle, total, idle):
    diff_idle = idle - prev_idle
    diff_total = total - prev_total
    cpu_used = diff_total-diff_idle
    cpu_usage = (cpu_used/diff_total)*100
    return cpu_usage

In [ ]:
cpu_data = {}
for device in devices:
    file = open(f'./{device}_cpu_profile.txt', 'r')
    text = file.read()
    stat_arr = text.split("###")
    
    cpu_percentages = {}
    prev_total = {}
    prev_idle = {}

    for stat in stat_arr:
        lines = stat.split('\n')
        for line in lines:
            if line.startswith('cpu'):
                cpu_info = line.split()
                total = int(cpu_info[1]) + int(cpu_info[2]) + int(cpu_info[3]) + int(cpu_info[4]) + int(cpu_info[5]) + int(cpu_info[6]) + int(cpu_info[7])
                idle = int(cpu_info[4])
                if line[3] != " ":
                    key = f"core {line[3]}"
                else:
                    key = "average"

                if cpu_percentages.get(key, None) == None:
                    cpu_percentages[key] = []
                    prev_total[key] = [total]
                    prev_idle[key] = [idle]

                else:
                    value = calculate_cpu_percentage(prev_total[key][-1], prev_idle[key][-1], total, idle)
                    cpu_percentages[key].append(value)
                    prev_total[key].append(total)
                    prev_idle[key].append(idle)
                    
    cpu_data[device] = pd.DataFrame.from_dict(cpu_percentages)

### 3.1 Time Series

In [ ]:
for device in devices:
    print(f"{device}:")
    df = cpu_data[device]
    for col in df.columns:
        sns.lineplot(df[col], label=col)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
    plt.show()

### 3.2 Average per core cpu usage

In [ ]:
for device in devices:
    print(f"{device}:")
    df = cpu_data[device]
    avg_core_usage = {}
    for col in df.columns:
        if col != "average":
            avg_core_usage[col] = np.mean(df[col])
    plt.figure(figsize=(8,6))
    sns.barplot(x=list(avg_core_usage.keys()), y=list(avg_core_usage.values()))
    plt.ylabel("percentage (%)")
    plt.show()

### 3.3 Average CPU usage

In [ ]:
avg_cpu_usage = {}
for device in devices:
    df = cpu_data[device]
    avg_cpu_usage[device] = np.mean(df["average"])

plt.figure(figsize=(8,6))
sns.barplot(x=list(avg_cpu_usage.keys()), y=list(avg_cpu_usage.values()))
plt.ylabel("percentage (%)")
plt.show()